# Ensembling of ConvNeXtLarge


# Base imports


In [ ]:
# Fix randomness and hide warnings
seed = 90

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np

import logging

import random

!pip install keras-cv tensorflow --upgrade

In [ ]:
# Import tensorflow
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras.applications import ConvNeXtLarge
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model, Sequential

import tensorflow as tf

from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten
import numpy as np

from tensorflow.keras import layers

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.15.0


In [ ]:
# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns

import keras_cv
import keras_core as keras
import numpy as np

Using TensorFlow backend


# Build the ensembled models

## Load dataset

In [ ]:
# Import the data
train = np.load('Scripts/AugmentationScript/MergedDatasetV2_custom_parameters/train_dataset_aug_mix_1.npz', allow_pickle=True)
validation = np.load('Scripts/AugmentationScript/MergedDatasetV2_custom_parameters/val_dataset.npz', allow_pickle=True)
test = np.load('Scripts/AugmentationScript/MergedDatasetV2_custom_parameters/test_dataset.npz', allow_pickle=True)

In [ ]:
# Divite into data and labels
X_train = train['data']
y_train = train['labels']

X_val = validation['data']
y_val = validation['labels']

X_test = test['data']
y_test = test['labels']

In [ ]:
# Display a sample of images from the training-validation dataset
num_img = 20
fig, axes = plt.subplots(1, num_img, figsize=(20,20))

# Iterate through the selected number of images
for i in range(num_img):
    # Select a random index
    idx = np.random.randint(0, len(X_train))

    ax = axes[i % num_img]
    # Display the normalized image using imshow
    ax.imshow(X_train[idx])
    ax.set_title({y_train[idx]})  # Show the corresponding digit label

# Adjust layout and display the images
plt.tight_layout()
plt.show()

In [ ]:
# Transformation of the labels from categorical to numerical

labels_dict = {'healthy': 0, 'unhealthy': 1}

y_train_array = []

for i in range(len(y_train)):
  y_train_array.append(labels_dict[tuple(y_train)[i]])
y_train = np.array(y_train_array)

y_val_array = []
for i in range(len(y_val)):
  y_val_array.append(labels_dict[tuple(y_val)[i]])
y_val = np.array(y_val_array)


y_test_array = []
for i in range(len(y_test)):
  y_test_array.append(labels_dict[tuple(y_test)[i]])
y_test = np.array(y_test_array)

# One-hot transformation of labels
y_train = tfk.utils.to_categorical(y_train)
y_val = tfk.utils.to_categorical(y_val)
y_test = tfk.utils.to_categorical(y_test)

In [ ]:
# Print the shapes of the sets
print("Training Data Shape:", X_train.shape)
print("Training Label Shape:", y_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Validation Label Shape:", y_val.shape)
print("Test Data Shape:", X_test.shape)
print("Test Label Shape:", y_test.shape)

Training Data Shape: (8253, 96, 96, 3)
Training Label Shape: (8253, 2)
Validation Data Shape: (872, 96, 96, 3)
Validation Label Shape: (872, 2)
Test Data Shape: (484, 96, 96, 3)
Test Label Shape: (484, 2)


In [ ]:
del train
del validation
del test

## Build the baseline model

In [ ]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 128                 # Batch size for training
epochs = 200                     # Number of training epochs

# Print the defined parameters
print("Epochs:", epochs)
print("Batch Size:", batch_size)
print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

Epochs: 200
Batch Size: 128
Input Shape: (96, 96, 3)
Output Shape: 2


## Ensembling

In [ ]:
import tensorflow.keras
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
keras_model = tensorflow.keras.models.load_model('Deliverables/SubmissionModels/model_1', compile=False)
keras_model._name = 'model1'
keras_model2 = tensorflow.keras.models.load_model('Deliverables/SubmissionModels/model_2', compile=False)
keras_model2._name = 'model2'
models = [keras_model, keras_model2]
#model_input = tf.keras.Input(shape=(125, 125, 3))
model_input = tf.keras.Input(shape=(96, 96, 3))
model_outputs = [model(model_input) for model in models]


ensemble_output = layers.average(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

ensemble_model.compile(optimizer=tfk.optimizers.AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 model1 (Sequential)         (None, 2)                    1966574   ['input_2[0][0]']             
                                                          74                                      
                                                                                                  
 model2 (Functional)         (None, 2)                    3933149   ['input_2[0][0]']             
                                                          48                                      
                                                                                              

In [ ]:

# Evaluate the model on the test set
score = ensemble_model.evaluate(X_test, y_test, verbose=0)

# Print test accuracy
print("Test accuracy:", score[1])

# plot the confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

# Predict the values from the validation dataset
Y_pred = ensemble_model.predict(X_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
ensemble_model.save("All_Colab_Scripts/ConvNeXtLarge/Ensembled/Ensembled_FinalModel")

In [ ]:
tfk.utils.plot_model(ensemble_model, expand_nested=True, show_shapes=True)

# Fine Tuning

In [ ]:
ConvNeXtLarge_model = tfk.models.load_model('All_Colab_Scripts/ConvNeXtLarge/Ensembled/Ensembled_FinalModel')


name = 'Ensembled_FinalModel'

ConvNeXtLarge_model.summary()

In [ ]:
# Enable one of the two models ensembled to perform fine tuning
trainable_layers = 15
for i, layer in enumerate(ConvNeXtLarge_model.layers[1].layers[(len(ConvNeXtLarge_model.layers[1].layers) - trainable_layers):]):
  layer.trainable=True # da 1 a N sono trainabili
for i, layer in enumerate(ConvNeXtLarge_model.layers[1].layers[:(len(ConvNeXtLarge_model.layers[1].layers) - trainable_layers)]):
  layer.trainable=False # da 1 a N non sono trainabili
for i, layer in enumerate(ConvNeXtLarge_model.layers[1].layers):
   print(i, layer.name, layer.trainable)

# Print the model summary
ConvNeXtLarge_model.summary()

In [ ]:
!pip install keras-cv tensorflow --upgrade
import keras_cv
import keras_core as keras
import numpy as np

ConvNeXtLarge_model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  #layers.Resizing(IMG_SIZE, IMG_SIZE),
  #keras_cv.layers.RandomSaturation((0.0, 0.5)),
  #layers.RandomTranslation(0.2, 0.2),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation((0.3, 0.5)),
  layers.RandomContrast((0.0, 0.3)),
  layers.RandomBrightness((0.0, 0.3)),
  layers.RandomZoom((-0.14, 0.14), (-0.14, 0.14)),
  #keras_cv.layers.RandomShear(.1, .1),
  #keras_cv.layers.RandAugment(value_range=(0, 1), augmentations_per_image=3, magnitude=0.3),

  # Rest of the model.
  ConvNeXtLarge_model
])



In [ ]:
ConvNeXtLarge_model.compile(optimizer=tfk.optimizers.AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model on augmented data, we choose to monitor only loss since the validtion set is not augmented and the model must improve only on augmented data in this phase
# Define early stopping callbacks
# Define the callbacks
callbacks = [
    tfk.callbacks.EarlyStopping(
        monitor="loss",
        patience=15,
        restore_best_weights=True,
    ),
    tfk.callbacks.ReduceLROnPlateau(
        monitor="loss",
        factor=0.5,
        patience=15,
        min_lr=1e-6,
        verbose=1,
    ),
]

# Fit the model
history = ConvNeXtLarge_model.fit(
    x=X_train,
    y=y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=callbacks
).history

# Save the trained model
ConvNeXtLarge_model.save("All_Colab_Scripts/ConvNeXtLarge/Fine_Tuned/ft_" + name)

# How many initial epochs to skip in the plot
begin_plot = 2

# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'][begin_plot:])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'][begin_plot:], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'][begin_plot:], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'][begin_plot:], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'][begin_plot:], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch+begin_plot], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

del history

del ConvNeXtLarge_model

In [ ]:
name = "All_Colab_Scripts/ConvNeXtLarge/Fine_Tuned/ft_Ensembled_FinalModel"

ConvNeXtLarge_model = tfk.models.load_model(name)

# Evaluate the model on the test set
score = ConvNeXtLarge_model.evaluate(X_test, y_test, verbose=0)

# Print test accuracy
print("Test accuracy:", score[1])

# plot the confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

# Predict the values from the validation dataset
Y_pred = ConvNeXtLarge_model.predict(X_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


# Re-Train on validation and train

## Build the baseline model

In [ ]:
# Import the data
train = np.load('Scripts/AugmentationScript/MergedDatasetV2_custom_parameters/train_val_dataset.npz', allow_pickle=True)

X_trainval = train['data']
y_trainval = train['labels']

labels_dict = {'healthy': 0, 'unhealthy': 1}

y_trainval_array = []

for i in range(len(y_trainval)):
  y_trainval_array.append(labels_dict[tuple(y_trainval)[i]])
y_trainval = np.array(y_trainval_array)

y_trainval = tfk.utils.to_categorical(y_trainval)

In [ ]:
# Define key model parameters
input_shape = X_trainval.shape[1:]  # Input shape for the model
output_shape = y_trainval.shape[1]  # Output shape for the model
batch_size = 64                 # Batch size for training
epochs = 200                     # Number of training epochs

# Print the defined parameters
print("Epochs:", epochs)
print("Batch Size:", batch_size)
print("Input Shape:", input_shape)
print("Output Shape:", output_shape)

## Train

In [ ]:
name = "All_Colab_Scripts/ConvNeXtLarge/Fine_Tuned/ft_Ensembled_FinalModel"

EfficientNetV2S_model = tfk.models.load_model(name)

EfficientNetV2S_model.summary()

In [ ]:

# V1

print(str(EfficientNetV2S_model.layers[5]))

model = Sequential()

model.add(EfficientNetV2S_model.layers[5])

model.summary()







In [ ]:
model1 = model.layers[0].layers[1]

model1.summary()

model2 = model.layers[0].layers[2]

model2.summary()


In [ ]:

trainable_layers = 15
for i, layer in enumerate(model1.layers[0].layers[(len(model1.layers[0].layers) - trainable_layers):]):
  layer.trainable=True # da 1 a N sono trainabili
for i, layer in enumerate(model1.layers[0].layers[:(len(model1.layers[0].layers) - trainable_layers)]):
  layer.trainable=False # da 1 a N non sono trainabili
for i, layer in enumerate(model1.layers[0].layers):
   print(i, layer.name, layer.trainable)

model1.layers[0].summary()

trainable_layers = 15
for i, layer in enumerate(model2.layers[0].layers[(len(model2.layers[0].layers) - trainable_layers):]):
  layer.trainable=True # da 1 a N sono trainabili
for i, layer in enumerate(model2.layers[0].layers[:(len(model2.layers[0].layers) - trainable_layers)]):
  layer.trainable=False # da 1 a N non sono trainabili
for i, layer in enumerate(model2.layers[0].layers):
   print(i, layer.name, layer.trainable)

model2.layers[0].summary()

model1.layers[0]._name = "ConvNeXtLarge_ft"

model2.layers[0]._name = "ConvNeXtLarge_cole"

models = [model1.layers[0], model2.layers[0]]
#model_input = tf.keras.Input(shape=(125, 125, 3))
model_input = tf.keras.Input(shape=(96, 96, 3))
model_outputs = [model(model_input) for model in models]


ensemble_output = layers.average(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output, name = "ensembled")

ensemble_model.compile(optimizer=tfk.optimizers.AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=tfk.optimizers.AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model.layers[0].name)

trainable_layers = 20
for i, layer in enumerate(model.layers[0].layers[2].layers[(len(model.layers[0].layers[2].layers) - trainable_layers):]):
  layer.trainable=True # da 1 a N sono trainabili
for i, layer in enumerate(model.layers[0].layers[2].layers[:(len(model.layers[0].layers[2].layers) - trainable_layers)]):
  layer.trainable=False # da 1 a N non sono trainabili
for i, layer in enumerate(model.layers[0].layers[2].layers):
   print(i, layer.name, layer.trainable)

model.layers[0].summary()

In [ ]:
del model

In [ ]:
# Define the callbacks
callbacks = [
    tfk.callbacks.EarlyStopping(
        monitor="loss",
        patience=10,
        restore_best_weights=True,
    ),
    tfk.callbacks.ReduceLROnPlateau(
        monitor="loss",
        factor=0.1,
        patience=3,
        min_lr=1e-6,
        verbose=1,
    ),
    tfk.callbacks.ModelCheckpoint(
        filepath="EfficientNetV2S_copilot/V2S_copilot_3.h5",
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1,
    ),
]

# Train the model
history = ensemble_model.fit(
    X_trainval,
    y_trainval,
    batch_size=64,
    epochs=30,
    callbacks=callbacks,
)

ensemble_model.save("All_Colab_Scripts/ConvNeXtLarge/Final/Ens_Leo_FinalModel")


In [ ]:
ensemble_model.save("All_Colab_Scripts/ConvNeXtLarge/Final/Ens_Leo_FinalModel")


# Improve futher the ensemble model

### Import ensembled

In [ ]:
ConvNeXtLarge_model = tfk.models.load_model('Deliverables/SubmissionModels/ConvNeXtLarge_Ensembled')

In [ ]:
ConvNeXtLarge_model.summary()

### Extract the ensembled

In [ ]:
ens_model = ConvNeXtLarge_model.layers[5]
ens_model.summary()

### Divide the models and finetune

In [ ]:
model1 = ens_model.layers[1]

model2 = ens_model.layers[2]

model1.summary()

model2.summary()

In [ ]:
# freeze layers of first model
trainable_layers = 0
for i, layer in enumerate(model1.layers[0].layers[(len(model1.layers[0].layers) - trainable_layers):]):
  layer.trainable=True # da 1 a N sono trainabili
for i, layer in enumerate(model1.layers[0].layers[:(len(model1.layers[0].layers) - trainable_layers)]):
  layer.trainable=False # da 1 a N non sono trainabili
for i, layer in enumerate(model1.layers[0].layers):
   print(i, layer.name, layer.trainable)


model1.layers[0].summary()


# unfreeze layers of second model
trainable_layers = 15
for i, layer in enumerate(model2.layers[0].layers[(len(model2.layers[0].layers) - trainable_layers):]):
  layer.trainable=True # da 1 a N sono trainabili
for i, layer in enumerate(model2.layers[0].layers[:(len(model2.layers[0].layers) - trainable_layers)]):
  layer.trainable=False # da 1 a N non sono trainabili
for i, layer in enumerate(model2.layers[0].layers):
   print(i, layer.name, layer.trainable)

model2.layers[0].summary()

In [ ]:
# merge the layers
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
keras_model = model1.layers[0]
keras_model._name = 'NonClasswise'
keras_model2 = model2.layers[0]
keras_model2._name = 'Classwise'
models = [keras_model, keras_model2]
#model_input = tf.keras.Input(shape=(125, 125, 3))
model_input = tf.keras.Input(shape=(96, 96, 3))
model_outputs = [model(model_input) for model in models]


ensemble_output = layers.average(model_outputs)
ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

ensemble_model.compile(optimizer=tfk.optimizers.AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

ensemble_model.summary()


In [ ]:

model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  #layers.Resizing(IMG_SIZE, IMG_SIZE),
  keras_cv.layers.RandomSaturation((0.0, 0.5)),
  layers.RandomTranslation(0.2, 0.2),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation((0.3, 0.5)),
  layers.RandomContrast((0.0, 0.3)),
  layers.RandomBrightness((0.0, 0.3)),
  layers.RandomZoom((-0.14, 0.14), (-0.14, 0.14)),
  keras_cv.layers.RandomShear(.1, .1),
  #keras_cv.layers.RandAugment(value_range=(0, 1), augmentations_per_image=3, magnitude=0.3),

  # Rest of the model.
  ensemble_model
])


In [ ]:
model.compile(optimizer=tfk.optimizers.AdamW(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#model = tfk.models.load_model('Deliverables/SubmissionModels/ConvNeXtLarge_Classwise')

In [ ]:
# train the model on augmented data, we choose to monitor only loss since the validtion set is not augmented and the model must improve only on augmented data in this phase
# Define early stopping callbacks
# Define the callbacks
callbacks = [
    tfk.callbacks.EarlyStopping(
        monitor="loss",
        patience=15,
        restore_best_weights=True,
    ),
    tfk.callbacks.ReduceLROnPlateau(
        monitor="loss",
        factor=0.5,
        patience=15,
        min_lr=1e-6,
        verbose=1,
    ),
]

# Fit the model
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=64,
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=callbacks
).history

# Save the trained model
model.save("All_Colab_Scripts/ConvNeXtLarge/Fine_Tuned/FT2_ENSEMBLED")

# How many initial epochs to skip in the plot
begin_plot = 2

# Find the epoch with the highest validation accuracy
best_epoch = np.argmax(history['val_accuracy'][begin_plot:])

# Plot training and validation performance metrics
plt.figure(figsize=(20, 5))

# Plot training and validation loss
plt.plot(history['loss'][begin_plot:], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_loss'][begin_plot:], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=0.3)

plt.figure(figsize=(20, 5))

# Plot training and validation accuracy, highlighting the best epoch
plt.plot(history['accuracy'][begin_plot:], label='Training', alpha=0.8, color='#ff7f0e', linewidth=3)
plt.plot(history['val_accuracy'][begin_plot:], label='Validation', alpha=0.8, color='#4D61E2', linewidth=3)
plt.plot(best_epoch, history['val_accuracy'][best_epoch+begin_plot], marker='*', alpha=0.8, markersize=10, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=0.3)

plt.show()

del history

#del ConvNeXtLarge_model

### Evaluation

In [ ]:
# Evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)

# Print test accuracy
print("Test accuracy:", score[1])

# plot the confusion matrix
from sklearn.metrics import confusion_matrix
import itertools

# Predict the values from the validation dataset
Y_pred = model.predict(X_test)
# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test, axis=1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)

# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()